In [2]:
import pandas as pd
import numpy as np

In [3]:
ccba = pd.read_csv('/home/bill/moneylaundring/private_dataset/private_x_ccba_full_hashed.csv')
cdtx = pd.read_csv('/home/bill/moneylaundring/private_dataset/private_x_cdtx0001_full_hashed.csv')
custinfo = pd.read_csv('/home/bill/moneylaundring/private_dataset/private_x_custinfo_full_hashed.csv')
dp = pd.read_csv('/home/bill/moneylaundring/private_dataset/private_x_dp_full_hashed.csv')
remit = pd.read_csv('/home/bill/moneylaundring/private_dataset/private_x_remit1_full_hashed.csv')
p_date = pd.read_csv('/home/bill/moneylaundring/private_dataset/private_x_alert_date.csv')
t_date = pd.read_csv('/home/bill/moneylaundring/dataset/train_x_alert_date.csv')
answer = pd.read_csv('/home/bill/moneylaundring/dataset/train_y_answer.csv')

In [4]:
ccba_merge = ccba.groupby('cust_id')
new_ccba = ccba_merge['cycam'].max()
new_ccba = pd.DataFrame(new_ccba)
new_ccba

,cycam
cust_id,
01b1735c3dbf0a35c1c989bb45434ce4326b365b73db0612190e56b2858e0184,120633.0
01c705b457297805c1c08137cb3adb164b8f393c265db2c3aff0a5484a703302,306239.0
035e2ebcd5c23f4dd0f1fc81a1fca8afd3121f7431bb585fa99c22e1a7164dba,1526090.0
0397056eb3652da244521539502ba1125c61c33c02a0084f9dae183bf6ae0508,411253.0
03e1bde277b33ccb68dc08cb120c761649622f44b0acc1fb284b7e1ef5ced626,20778.0
...,...
fd0ae9a4d32233ea311bd5ac65ed70bee5f1410f6e4d016602ef48f2cea62c0b,208728.0
fd18e359b0e0f16ba1f16c5aa76e5ef76064793b0ee00702a7018f53cbb41db3,824010.0
fd4620b0e018b626a0a5094bec42111466f2039fb8ea36a3090dcdeadc8389b0,104251.0


In [5]:
cinfo_test = custinfo.merge(new_ccba, on='cust_id',how='outer')

In [6]:
cinfo_test['cycam'].fillna(0,inplace=True)

In [7]:
cinfo_test.isna().any()

alert_key          False
cust_id            False
risk_rank          False
occupation_code    False
total_asset        False
AGE                False
cycam              False
dtype: bool

In [8]:
cinfo_test = cinfo_test.merge(p_date, on='alert_key')

In [9]:
cdtx_test = pd.merge(cinfo_test, cdtx, left_on=['cust_id','date'], right_on=['cust_id','date'])
dp_test = pd.merge(cinfo_test, dp, left_on=['cust_id','date'], right_on=['cust_id','tx_date'])
remit_test = pd.merge(cinfo_test, remit, left_on=['cust_id','date'], right_on=['cust_id','trans_date'])

In [10]:
def db_cr2num(row):
    return 0 if row == 'DB' else 1

In [11]:
dp_test['debit_credit'] = dp_test['debit_credit'].apply(db_cr2num)
dp_test['fiscTxId'].fillna(-1,inplace=True)
dp_test['txbranch'].fillna(-1,inplace=True)

In [12]:
cdtx_test_new = cdtx_test[['alert_key','risk_rank','occupation_code','total_asset','AGE','country','cur_type','amt','cycam','date']]
dp_test_new = dp_test[['alert_key','risk_rank','occupation_code','total_asset','AGE','debit_credit','tx_time','tx_type','tx_amt','exchg_rate','info_asset_code','fiscTxId','txbranch','cross_bank','ATM','cycam','date']]
remit_test_new = remit_test[['alert_key','risk_rank','occupation_code','total_asset','AGE','trans_no','trade_amount_usd','cycam','date']]

In [13]:
cdtx_test_new.isna().any()

alert_key          False
risk_rank          False
occupation_code    False
total_asset        False
AGE                False
country            False
cur_type           False
amt                False
cycam              False
date               False
dtype: bool

In [15]:
dp_test_new.isna().any()

alert_key          False
risk_rank          False
occupation_code    False
total_asset        False
AGE                False
debit_credit       False
tx_time            False
tx_type            False
tx_amt              True
exchg_rate         False
info_asset_code    False
fiscTxId           False
txbranch           False
cross_bank         False
ATM                False
cycam              False
date               False
dtype: bool

In [16]:
dp_test_new['tx_amt'].fillna(0,inplace=True)

/tmp/ipykernel_13002/204289302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dp_test_new['tx_amt'].fillna(0,inplace=True)


In [17]:
remit_test_new.isna().any()

alert_key           False
risk_rank           False
occupation_code     False
total_asset         False
AGE                 False
trans_no            False
trade_amount_usd    False
cycam               False
date                False
dtype: bool

In [18]:
cdtx_test_new.to_csv('/home/bill/moneylaundring/contest/processed_dataset/cdtx_test_private.csv',index=False)

In [19]:
dp_test_new.to_csv('/home/bill/moneylaundring/contest/processed_dataset/dp_test_private.csv',index=False)
remit_test_new.to_csv('/home/bill/moneylaundring/contest/processed_dataset/remit_test_private.csv',index=False)